In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warnings to keep output clean
warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# Define the folder where all the CSVs live
# -------------------------------------------------------------------
data_dir = r"C:\Users\sjn17\Downloads\mobile_app_analytics\data"

# Small helper to safely load CSVs
def load_csv_file(path):
    if os.path.exists(path):
        df = pd.read_csv(path)
        print(f"Loaded {os.path.basename(path)} → {df.shape[0]} rows, {df.shape[1]} cols")
        return df
    else:
        print(f" Could not find file: {os.path.basename(path)}")
        return None

# -------------------------------------------------------------------
# Load DAU (Daily Active Users) result files
# -------------------------------------------------------------------
dau_files = {}
for i in range(4):
    filename = f"dau_{i}results.csv"
    full_path = os.path.join(data_dir, filename)
    dau_files[f"dau_{i}"] = load_csv_file(full_path)

# -------------------------------------------------------------------
# Load Retention result files
# -------------------------------------------------------------------
retention_files = {}
for i in range(5):
    filename = f"retention_{i}results.csv"
    full_path = os.path.join(data_dir, filename)
    retention_files[f"retention_{i}"] = load_csv_file(full_path)

# -------------------------------------------------------------------
# Load Cohort analysis results
# -------------------------------------------------------------------
cohort_path = os.path.join(data_dir, "cohort_results.csv")
cohort_df = load_csv_file(cohort_path)

# -------------------------------------------------------------------
# Quick summary
# -------------------------------------------------------------------
print("\nSummary of loaded files:")
print(f"DAU files loaded: {sum(1 for df in dau_files.values() if df is not None)} / {len(dau_files)}")
print(f"Retention files loaded: {sum(1 for df in retention_files.values() if df is not None)} / {len(retention_files)}")
if cohort_df is not None:
    print(f"Cohort analysis file loaded successfully with {cohort_df.shape[0]} rows")


Loaded dau_0results.csv → 61 rows, 2 cols
Loaded dau_1results.csv → 61 rows, 6 cols
Loaded dau_2results.csv → 61 rows, 3 cols
Loaded dau_3results.csv → 1 rows, 5 cols
Loaded retention_0results.csv → 1 rows, 5 cols
Loaded retention_1results.csv → 61 rows, 4 cols
Loaded retention_2results.csv → 61 rows, 4 cols
Loaded retention_3results.csv → 61 rows, 4 cols
Loaded retention_4results.csv → 61 rows, 4 cols
Loaded cohort_results.csv → 61 rows, 6 cols

Summary of loaded files:
DAU files loaded: 4 / 4
Retention files loaded: 5 / 5
Cohort analysis file loaded successfully with 61 rows


Data Exploration and Understanding

In [3]:
# -------------------------------------------------------------------
# Explore the structure of the loaded data
# -------------------------------------------------------------------

# --- DAU files ---
print("\n=== Daily Active Users (DAU) Files ===")
for name, df in dau_files.items():
    if df is not None:
        print(f"\n{name}")
        print(f"Columns: {list(df.columns)}")
        print("Sample rows:")
        print(df.head(3))   # show first 3 rows
    else:
        print(f"\n{name} is missing")

# --- Retention files ---
print("\n=== Retention Files ===")
for name, df in retention_files.items():
    if df is not None:
        print(f"\n{name}")
        print(f"Columns: {list(df.columns)}")
        print("Sample rows:")
        print(df.head(3))
    else:
        print(f"\n{name} is missing")

# --- Cohort analysis file ---
print("\n=== Cohort Analysis File ===")
if cohort_df is not None:
    print(f"Columns: {list(cohort_df.columns)}")
    print("Sample rows:")
    print(cohort_df.head(5))   # show first 5 rows
else:
    print("Cohort file is missing")



=== Daily Active Users (DAU) Files ===

dau_0
Columns: ['date', 'dau']
Sample rows:
         date   dau
0  2025-07-03  2189
1  2025-07-04  2203
2  2025-07-05  2834

dau_1
Columns: ['date', 'dau', 'total_sessions', 'avg_session_duration', 'total_screens_viewed', 'avg_screens_per_session']
Sample rows:
         date   dau  total_sessions  avg_session_duration  \
0  2025-07-03  2189            2189             72.073261   
1  2025-07-04  2203            2203             73.048642   
2  2025-07-05  2834            2834             76.144241   

   total_screens_viewed  avg_screens_per_session  
0                 80124                36.603015  
1                 81678                37.075806  
2                109110                38.500353  

dau_2
Columns: ['date', 'dau', 'activity_level']
Sample rows:
         date   dau activity_level
0  2025-08-02  3015       Peak Day
1  2025-08-03  2971    Regular Day
2  2025-07-12  2926    Regular Day

dau_3
Columns: ['total_days', 'min_dau', 'ma

Data Cleaning and Validation

In [4]:
# -------------------------------------------------------------------
# Clean Daily Active Users (DAU) data
# -------------------------------------------------------------------
print("\n=== Cleaning DAU Data ===")
for name, df in dau_files.items():
    if df is None:
        print(f"{name} is missing, skipping...")
        continue

    print(f"\nCleaning {name}")
    print(f"Original shape: {df.shape}")

    # Check missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print("Missing values:")
        print(missing[missing > 0])
    else:
        print("No missing values found")

    # Convert date columns
    date_columns = [col for col in df.columns if "date" in col.lower()]
    for col in date_columns:
        try:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            print(f"Converted {col} to datetime")
        except Exception as e:
            print(f"Could not convert {col}: {e}")

    # Drop duplicates
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    dropped = before - df.shape[0]
    if dropped > 0:
        print(f"Removed {dropped} duplicate rows")

    print(f"Cleaned shape: {df.shape}")


=== Cleaning DAU Data ===

Cleaning dau_0
Original shape: (61, 2)
No missing values found
Converted date to datetime
Cleaned shape: (61, 2)

Cleaning dau_1
Original shape: (61, 6)
No missing values found
Converted date to datetime
Cleaned shape: (61, 6)

Cleaning dau_2
Original shape: (61, 3)
No missing values found
Converted date to datetime
Cleaned shape: (61, 3)

Cleaning dau_3
Original shape: (1, 5)
No missing values found
Cleaned shape: (1, 5)


In [5]:
# -------------------------------------------------------------------
# Clean Retention data
# -------------------------------------------------------------------
print("\n=== Cleaning Retention Data ===")
for name, df in retention_files.items():
    if df is None:
        print(f"{name} is missing, skipping...")
        continue

    print(f"\nCleaning {name}")
    print(f"Original shape: {df.shape}")

    # Missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print("Missing values:")
        print(missing[missing > 0])
    else:
        print("No missing values found")

    # Convert date columns
    date_columns = [col for col in df.columns if "date" in col.lower()]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        print(f"Converted {col} to datetime")

    # Drop duplicates
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    dropped = before - df.shape[0]
    if dropped > 0:
        print(f"Removed {dropped} duplicate rows")

    print(f"Cleaned shape: {df.shape}")



=== Cleaning Retention Data ===

Cleaning retention_0
Original shape: (1, 5)
No missing values found
Cleaned shape: (1, 5)

Cleaning retention_1
Original shape: (61, 4)
No missing values found
Converted first_date to datetime
Cleaned shape: (61, 4)

Cleaning retention_2
Original shape: (61, 4)
No missing values found
Converted first_date to datetime
Cleaned shape: (61, 4)

Cleaning retention_3
Original shape: (61, 4)
No missing values found
Converted first_date to datetime
Cleaned shape: (61, 4)

Cleaning retention_4
Original shape: (61, 4)
No missing values found
Converted first_date to datetime
Cleaned shape: (61, 4)


In [6]:
# -------------------------------------------------------------------
# Clean Cohort data
# -------------------------------------------------------------------
print("\n=== Cleaning Cohort Data ===")
if cohort_df is not None:
    print(f"Original shape: {cohort_df.shape}")

    # Missing values
    missing = cohort_df.isnull().sum()
    if missing.sum() > 0:
        print("Missing values:")
        print(missing[missing > 0])
    else:
        print("No missing values found")

    # Convert date columns
    date_columns = [col for col in cohort_df.columns if "date" in col.lower()]
    for col in date_columns:
        cohort_df[col] = pd.to_datetime(cohort_df[col], errors="coerce")
        print(f"Converted {col} to datetime")

    # Drop duplicates
    before = cohort_df.shape[0]
    cohort_df.drop_duplicates(inplace=True)
    dropped = before - cohort_df.shape[0]
    if dropped > 0:
        print(f"Removed {dropped} duplicate rows")

    print(f"Cleaned shape: {cohort_df.shape}")
else:
    print("Cohort data missing, skipping...")


=== Cleaning Cohort Data ===
Original shape: (61, 6)
No missing values found
Converted cohort_date to datetime
Cleaned shape: (61, 6)


Calculate Basic Statistics

In [47]:
# --- DAU Statistics ---
from tabulate import tabulate
print("\n=== DAU Statistics ===")
for name, df in dau_files.items():
    if df is None or df.empty:
        print(f"{name} missing or empty, skipping...")
        continue

    print(f"\n{name}")
    print(f"Shape: {df.shape}")

    # --- Numeric stats as table ---
    if not df.select_dtypes(include=np.number).empty:
        num_summary = df.describe().T
        print("\nNumeric columns summary:")
        print(tabulate(num_summary, headers="keys", tablefmt="pretty", floatfmt=".2f"))

    # --- Categorical stats as table ---
    cat_cols = df.select_dtypes(include="object").columns
    if len(cat_cols) > 0:
        cat_table = [[col, df[col].nunique()] for col in cat_cols]
        print("\nCategorical columns overview:")
        print(tabulate(cat_table, headers=["Column", "Unique Values"], tablefmt="pretty"))

    # --- Date range as table ---
    date_cols = [col for col in df.columns if "date" in col.lower()]
    date_table = []
    for col in date_cols:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            date_table.append([col, df[col].min(), df[col].max()])
    if date_table:
        print("\nDate range overview:")
        print(tabulate(date_table, headers=["Date Column", "Min Date", "Max Date"], tablefmt="pretty"))



=== DAU Statistics ===

dau_0
Shape: (61, 2)

Numeric columns summary:
+------+-------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+
|      | count |        mean         |         min         |         25%         |         50%         |         75%         |         max         |        std        |
+------+-------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+
| date |  61   | 2025-08-02 00:00:00 | 2025-07-03 00:00:00 | 2025-07-18 00:00:00 | 2025-08-02 00:00:00 | 2025-08-17 00:00:00 | 2025-09-01 00:00:00 |        nan        |
| dau  | 61.0  |  2396.622950819672  |       2025.0        |       2191.0        |       2254.0        |       2763.0        |       3015.0        | 303.3529387767339 |
+------+-------+---------------------+---------------------+---------------------+-

In [48]:
print("\n=== Retention Statistics ===")
from tabulate import tabulate

for name, df in retention_files.items():
    if df is None or df.empty:
        print(f"{name}:Missing or empty")
        continue

    print(f"\n{name}")
    print(f"Shape: {df.shape}")

    # Numeric columns summary (as table)
    num_summary = df.describe().T  # transpose so rows = columns
    print("\nNumeric columns summary:")
    print(tabulate(num_summary, headers="keys", tablefmt="pretty", floatfmt=".2f"))



=== Retention Statistics ===

retention_0
Shape: (1, 5)

Numeric columns summary:
+-------------------+-------+----------+-----+----------+----------+----------+----------+----------+
|                   | count |   mean   | std |   min    |   25%    |   50%    |   75%    |   max    |
+-------------------+-------+----------+-----+----------+----------+----------+----------+----------+
|    total_users    |  1.0  | 259786.0 | nan | 259786.0 | 259786.0 | 259786.0 | 259786.0 | 259786.0 |
|  weekly_retained  |  1.0  | 253709.0 | nan | 253709.0 | 253709.0 | 253709.0 | 253709.0 | 253709.0 |
| monthly_retained  |  1.0  | 258996.0 | nan | 258996.0 | 258996.0 | 258996.0 | 258996.0 | 258996.0 |
| weekly_retention  |  1.0  |  97.66   | nan |  97.66   |  97.66   |  97.66   |  97.66   |  97.66   |
| monthly_retention |  1.0  |   99.7   | nan |   99.7   |   99.7   |   99.7   |   99.7   |   99.7   |
+-------------------+-------+----------+-----+----------+----------+----------+----------+----------+

In [49]:
# --- Cohort Statistics ---
print("\n=== Cohort Statistics ===")
from tabulate import tabulate

if cohort_df is not None and not cohort_df.empty:
    print(f"Shape: {cohort_df.shape}")

    # Numeric stats in tabular form
    if not cohort_df.select_dtypes(include=np.number).empty:
        num_summary = cohort_df.describe().T
        print("\nNumeric columns summary:")
        print(tabulate(num_summary, headers="keys", tablefmt="pretty", floatfmt=".2f"))

    # Categorical overview
    cat_cols = cohort_df.select_dtypes(include="object").columns
    if len(cat_cols) > 0:
        print("\nCategorical columns overview:")
        cat_table = [[col, cohort_df[col].nunique()] for col in cat_cols]
        print(tabulate(cat_table, headers=["Column", "Unique Values"], tablefmt="pretty"))

    # Date range check
    date_cols = [col for col in cohort_df.columns if "date" in col.lower()]
    date_table = []
    for col in date_cols:
        if pd.api.types.is_datetime64_any_dtype(cohort_df[col]):
            date_table.append([col, cohort_df[col].min(), cohort_df[col].max()])
    if date_table:
        print("\nDate range overview:")
        print(tabulate(date_table, headers=["Date Column", "Min Date", "Max Date"], tablefmt="pretty"))

else:
    print("Cohort data missing, skipping...")



=== Cohort Statistics ===
Shape: (61, 6)

Numeric columns summary:
+------------------+-------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+--------------------+
|                  | count |        mean         |         min         |         25%         |         50%         |         75%         |         max         |        std         |
+------------------+-------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+--------------------+
|   cohort_date    |  61   | 2025-08-02 00:00:00 | 2025-07-03 00:00:00 | 2025-07-18 00:00:00 | 2025-08-02 00:00:00 | 2025-08-17 00:00:00 | 2025-09-01 00:00:00 |        nan         |
|   total_users    | 61.0  | 153.11475409836066  |         7.0         |        14.0         |        43.0         |        112.0        |       2189.0        | 355.70137748588377 |
|   d1_retention   | 6

Combine and Consolidate Data

In [50]:
import pandas as pd

# -------------------------------------------------------------------
# Combine and Consolidate Data + Tabular Summary
# -------------------------------------------------------------------

results = []  # to store summary rows

# --- Combine DAU Data ---
print("\n=== Combining DAU Data ===")
dau_combined = pd.DataFrame()
for name, df in dau_files.items():
    if df is None or df.empty:
        results.append(["DAU", name, "Skipped (empty/missing)", (0, 0)])
        continue

    df = df.copy()
    df["source_file"] = name
    if dau_combined.empty:
        dau_combined = df
        results.append(["DAU", name, "Combined (first file)", df.shape])
    else:
        if set(df.columns) - {"source_file"} == set(dau_combined.columns) - {"source_file"}:
            dau_combined = pd.concat([dau_combined, df], ignore_index=True)
            results.append(["DAU", name, "Combined", df.shape])
        else:
            results.append(["DAU", name, "Different structure", df.shape])

# --- Combine Retention Data ---
print("\n=== Combining Retention Data ===")
retention_combined = pd.DataFrame()
for name, df in retention_files.items():
    if df is None or df.empty:
        results.append(["Retention", name, "Skipped (empty/missing)", (0, 0)])
        continue

    df = df.copy()
    df["source_file"] = name
    if retention_combined.empty:
        retention_combined = df
        results.append(["Retention", name, "Combined (first file)", df.shape])
    else:
        if set(df.columns) - {"source_file"} == set(retention_combined.columns) - {"source_file"}:
            retention_combined = pd.concat([retention_combined, df], ignore_index=True)
            results.append(["Retention", name, "Combined", df.shape])
        else:
            results.append(["Retention", name, "Different structure", df.shape])

# -------------------------------------------------------------------
# Show results in a clean table
# -------------------------------------------------------------------
summary_df = pd.DataFrame(results, columns=["Dataset", "File", "Status", "Shape"])

try:
    from IPython.display import display
    display(summary_df)
except ImportError:
    # Pretty table in terminal
    from tabulate import tabulate
    print("\n=== Combination Summary ===")
    print(tabulate(summary_df, headers="keys", tablefmt="pretty"))

# Final combined shapes
print("\nFinal Combined Shapes:")
print(f"- DAU Combined Shape: {dau_combined.shape}")
print(f"- Retention Combined Shape: {retention_combined.shape}")



=== Combining DAU Data ===

=== Combining Retention Data ===


,Dataset,File,Status,Shape
0,DAU,dau_0,Combined (first file),"(61, 3)"
1,DAU,dau_1,Different structure,"(61, 7)"
2,DAU,dau_2,Different structure,"(61, 4)"
3,DAU,dau_3,Different structure,"(1, 6)"
4,Retention,retention_0,Combined (first file),"(1, 6)"
5,Retention,retention_1,Different structure,"(61, 5)"
6,Retention,retention_2,Different structure,"(61, 5)"
7,Retention,retention_3,Different structure,"(61, 5)"
8,Retention,retention_4,Different structure,"(61, 5)"



Final Combined Shapes:
- DAU Combined Shape: (61, 3)
- Retention Combined Shape: (1, 6)


Data Validation Checks

In [51]:
import pandas as pd
import numpy as np

print("=== Data Validation ===")

validation_results = []

# --- DAU checks ---
if not dau_combined.empty:
    # Date ranges
    date_cols = [col for col in dau_combined.columns if dau_combined[col].dtype == 'datetime64[ns]']
    for col in date_cols:
        validation_results.append([
            "DAU", col, "Date Range",
            f"{dau_combined[col].min().date()} → {dau_combined[col].max().date()}"
        ])

    # Negative values
    numeric_cols = dau_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if (dau_combined[col] < 0).any():
            negative_count = (dau_combined[col] < 0).sum()
            validation_results.append(["DAU", col, "Negative Values", f"{negative_count} rows"])
        else:
            validation_results.append(["DAU", col, "Negative Values", "None"])

# --- Retention checks ---
if not retention_combined.empty:
    date_cols = [col for col in retention_combined.columns if retention_combined[col].dtype == 'datetime64[ns]']
    for col in date_cols:
        validation_results.append([
            "Retention", col, "Date Range",
            f"{retention_combined[col].min().date()} → {retention_combined[col].max().date()}"
        ])

    numeric_cols = retention_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if (retention_combined[col] < 0).any():
            negative_count = (retention_combined[col] < 0).sum()
            validation_results.append(["Retention", col, "Negative Values", f"{negative_count} rows"])
        else:
            validation_results.append(["Retention", col, "Negative Values", "None"])

# --- Cohort checks ---
if cohort_df is not None and not cohort_df.empty:
    date_cols = [col for col in cohort_df.columns if cohort_df[col].dtype == 'datetime64[ns]']
    for col in date_cols:
        validation_results.append([
            "Cohort", col, "Date Range",
            f"{cohort_df[col].min().date()} → {cohort_df[col].max().date()}"
        ])

    numeric_cols = cohort_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if (cohort_df[col] < 0).any():
            negative_count = (cohort_df[col] < 0).sum()
            validation_results.append(["Cohort", col, "Negative Values", f"{negative_count} rows"])
        else:
            validation_results.append(["Cohort", col, "Negative Values", "None"])

# --- Convert to DataFrame ---
validation_df = pd.DataFrame(validation_results, columns=["Dataset", "Column", "Check", "Result"])
print(validation_df.to_string(index=False))


=== Data Validation ===
  Dataset            Column           Check                  Result
      DAU              date      Date Range 2025-07-03 → 2025-09-01
      DAU               dau Negative Values                    None
Retention       total_users Negative Values                    None
Retention   weekly_retained Negative Values                    None
Retention  monthly_retained Negative Values                    None
Retention  weekly_retention Negative Values                    None
Retention monthly_retention Negative Values                    None
   Cohort       cohort_date      Date Range 2025-07-03 → 2025-09-01
   Cohort       total_users Negative Values                    None
   Cohort      d1_retention Negative Values                    None
   Cohort   week1_retention Negative Values                    None
   Cohort  month1_retention Negative Values                    None
   Cohort    rest_retention Negative Values                    None


In [53]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# Data Validation Checks
# -------------------------------------------------------------------
validation_results = []

# --- DAU Validation ---
if not dau_combined.empty:
    # Date range check
    date_cols = [col for col in dau_combined.columns if pd.api.types.is_datetime64_any_dtype(dau_combined[col])]
    for col in date_cols:
        validation_results.append({
            "Dataset": "DAU",
            "Column": col,
            "Check": "Date Range",
            "Result": f"{dau_combined[col].min().date()} → {dau_combined[col].max().date()}"
        })

    # Negative values check
    numeric_cols = dau_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        negative_count = (dau_combined[col] < 0).sum()
        validation_results.append({
            "Dataset": "DAU",
            "Column": col,
            "Check": "Negative Values",
            "Result": "None" if negative_count == 0 else f"{negative_count} negatives"
        })

# --- Retention Validation ---
if not retention_combined.empty:
    numeric_cols = retention_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        negative_count = (retention_combined[col] < 0).sum()
        validation_results.append({
            "Dataset": "Retention",
            "Column": col,
            "Check": "Negative Values",
            "Result": "None" if negative_count == 0 else f"{negative_count} negatives"
        })

# --- Cohort Validation ---
if cohort_df is not None:
    # Date range check
    date_cols = [col for col in cohort_df.columns if pd.api.types.is_datetime64_any_dtype(cohort_df[col])]
    for col in date_cols:
        validation_results.append({
            "Dataset": "Cohort",
            "Column": col,
            "Check": "Date Range",
            "Result": f"{cohort_df[col].min().date()} → {cohort_df[col].max().date()}"
        })

    # Negative values check
    numeric_cols = cohort_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        negative_count = (cohort_df[col] < 0).sum()
        validation_results.append({
            "Dataset": "Cohort",
            "Column": col,
            "Check": "Negative Values",
            "Result": "None" if negative_count == 0 else f"{negative_count} negatives"
        })

# -------------------------------------------------------------------
# Display as a nice Pandas styled table
# -------------------------------------------------------------------
validation_df = pd.DataFrame(validation_results)

pd.set_option("display.colheader_justify", "center")

display(
    validation_df.style
        .set_table_attributes("style='display:inline'")
        .set_caption("=== Data Validation ===")
        .hide(axis="index")  # hides row index for cleaner look
)


Dataset,Column,Check,Result
DAU,date,Date Range,2025-07-03 → 2025-09-01
DAU,dau,Negative Values,None
Retention,total_users,Negative Values,None
Retention,weekly_retained,Negative Values,None
Retention,monthly_retained,Negative Values,None
Retention,weekly_retention,Negative Values,None
Retention,monthly_retention,Negative Values,None
Cohort,cohort_date,Date Range,2025-07-03 → 2025-09-01
Cohort,total_users,Negative Values,None
Cohort,d1_retention,Negative Values,None


Create Final Clean Datasets

In [57]:
# -------------------------------------------------------------------
# Create Final Clean Dataset (Merged without 'source_file')
# -------------------------------------------------------------------
print("=== Creating Final Clean Dataset ===")

final_datasets = []

# --- DAU Data ---
if not dau_combined.empty:
    dau_temp = dau_combined.copy()
    if "source_file" in dau_temp.columns:
        dau_temp.drop(columns=["source_file"], inplace=True)
    dau_temp["Dataset"] = "DAU"
    final_datasets.append(dau_temp)

# --- Retention Data ---
if not retention_combined.empty:
    retention_temp = retention_combined.copy()
    if "source_file" in retention_temp.columns:
        retention_temp.drop(columns=["source_file"], inplace=True)
    retention_temp["Dataset"] = "Retention"
    final_datasets.append(retention_temp)

# --- Cohort Data ---
if cohort_df is not None and not cohort_df.empty:
    cohort_temp = cohort_df.copy()
    if "source_file" in cohort_temp.columns:
        cohort_temp.drop(columns=["source_file"], inplace=True)
    cohort_temp["Dataset"] = "Cohort"
    final_datasets.append(cohort_temp)

# --- Merge all datasets ---
if final_datasets:
    final_clean_df = pd.concat(final_datasets, ignore_index=True, sort=False)
    final_clean_df.to_csv("final_clean_dataset.csv", index=False)

    print("✅ Saved final_clean_dataset.csv")
    print("\n=== Preview of Final Clean Dataset ===")
    display(final_clean_df.head(20).style.set_caption("📊 Final Clean Dataset (Preview)"))
else:
    print("⚠️ No data available to create final dataset.")


=== Creating Final Clean Dataset ===
✅ Saved final_clean_dataset.csv

=== Preview of Final Clean Dataset ===


,date,dau,Dataset,total_users,weekly_retained,monthly_retained,weekly_retention,monthly_retention,cohort_date,d1_retention,week1_retention,month1_retention,rest_retention
0,2025-07-03 00:00:00,2189.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
1,2025-07-04 00:00:00,2203.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
2,2025-07-05 00:00:00,2834.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
3,2025-07-06 00:00:00,2861.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
4,2025-07-07 00:00:00,2202.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
5,2025-07-08 00:00:00,2225.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
6,2025-07-09 00:00:00,2214.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
7,2025-07-10 00:00:00,2120.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
8,2025-07-11 00:00:00,2267.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
9,2025-07-12 00:00:00,2926.000000,DAU,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan
